<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Требуется защитить данные клиентов страховой компании: разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
# загружаю необходимые библиотеки
import pandas as pd
import numpy as np
import random 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [3]:
print('Уникальные значения Пол', data['Пол'].unique())
print('Уникальные значения Возраст', sorted(data['Возраст'].unique()))
print('Уникальные значения Члены семьи', sorted(data['Члены семьи'].unique()))
print('Уникальные значения Страховые выплаты', sorted(data['Страховые выплаты'].unique()))

Уникальные значения Пол [1 0]
Уникальные значения Возраст [18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 65.0]
Уникальные значения Члены семьи [0, 1, 2, 3, 4, 5, 6]
Уникальные значения Страховые выплаты [0, 1, 2, 3, 4, 5]


In [4]:
print(data.isnull().sum())

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [5]:
print(data.duplicated().sum())

153


In [6]:
display(data[data.duplicated()])

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


In [7]:
data = data.drop_duplicates()
print(data.shape)

(4847, 5)


In [8]:
# создаю обучающие и целевой признаки
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

print(features.shape)
print(target.shape)

(4847, 4)
(4847,)


***Вывод***

В данных нет пропусков, но есть 153 дубликата, эти строки были удалены из общего датасета. Были созданы обучабщие и целевой признаки. Данные готовы для дальнейшей работы.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

In [9]:
# умножаю транспонентную матрицу признаков на первоначальную матрицу
class LinearRegressionOriginal:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target.values
        w = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegressionOriginal()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print(r2)

0.4302010044852068


In [15]:
class LinearRegressionCrypt:
    def fit(self, train_features, train_target):
        self.P = np.random.normal(size=(train_features.shape[1],train_features.shape[1]))
        train_features = train_features @ self.P
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target.values
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        test_features = test_features @ self.P
        return test_features.dot(self.w) + self.w0
    
model = LinearRegressionCrypt()
model.fit(features, target)
predictions = model.predict(features)
r2_P = r2_score(target, predictions)
print(r2_P)

0.43020100448520393


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** b. Значение r2 не изменилось 

**Обоснование:** 

Использую основные свойства матриц:
$ A E = E A = A $

$ A A^{-1} = E $

$ (AB)^T = B^TA^T $

$ (AB)^{-1} = B^{-1}A^{-1} $

$ (AB)C = A(BC) $

***

По первому условию $w = (X^T X)^{-1} X^T y$. Распишу $w$:

$$ w = (X^T X)^{-1} X^T y =  X^{-1} (X^T)^{-1} X^T y = X^{-1} E y = X^{-1} y$$ 

Найду новое $w'$, для которого вместо $X$ использую $X'=XP$.


$$w' = ((X')^T  (X'))^{-1}  (X')^T  y$$

$$w' = ((XP)^T (XP))^{-1}  (XP)^T  y$$

$$w' =  (XP)^{-1} ((XP)^T)^{-1} (XP)^T  y$$

Так как $ A A^{-1} = E $, то $ ((XP)^T)^{-1} (XP)^T = E $
$$w' =  (XP)^{-1} E  y$$

Так как $ A E = E A = A $, то $ (XP)^{-1} E = (XP)^{-1} $
$$w' =  X^{-1}P^{-1} E  y$$
$$w' =  X^{-1}P^{-1}  y$$
$$w' =  wP^{-1}$$

Тогда $a = Xw$, а новые $a' = X'w' = XP wP^{-1} = XwE = Xw = a $

А так как $a = a'$, то и значение r2 не изменяется.

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$


$$
w' = (P^T ((X^T X) P))^{-1} (XP)^T y
$$
$$
w' = ((X^T X) P))^{-1}(P^T)^{-1}  (XP)^T y
$$
$$
w' = ((X^T X) P))^{-1}(P^T)^{-1}  P^T X^T y
$$
$$
w' = ((X^T X) P))^{-1}E X^T y
$$
$$
w' = ((X^T X) P))^{-1} X^T y
$$
$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$

$$
w' = P^{-1}(X^T X)^{-1} X^T y = P^{-1} w
$$

$$
a = (XP)w' = (XP) P^{-1} w = Xw
$$

## Алгоритм преобразования

**Алгоритм**

Чтобы было сложно восстановить персональную информацию, необходимо признаки умножить на рандомную нормальную матрицу

**Обоснование**

Исходя из пункта выше, выяснили, что при умножении исходной матрицы признаков на рандомную обратимую матрицу, качество линейной регрессии остается прежним за счет того, что предсказания не изменяются. 

## Проверка алгоритма

In [16]:
# создаю обучающие и целевой признаки, умножая исходные на рандомную матрицу
random_matrix= np.random.normal(size=(features.shape[1],features.shape[1]))

if  np.linalg.det(random_matrix) != 0:
    print('У матрицы имеется обратная')
else:
    print('У матрицы нет обратной')
print()
new_features = features @ random_matrix
new_target = target 

print(random_matrix)
print(new_features.shape)
print(new_target.shape)

У матрицы имеется обратная

[[ 0.38210443  0.49606684  0.80620974 -0.12501344]
 [ 0.71446475 -0.49730825  0.37714312 -1.24517535]
 [-1.08209504  0.644921   -0.75796115  0.6088718 ]
 [-1.37654638  0.77491964  2.22218912 -0.68066225]]
(4847, 4)
(4847,)


In [17]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target.values
        w = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(new_features, new_target)
new_predictions = model.predict(new_features)
new_r2 = r2_score(new_target, new_predictions)
print(new_r2)

0.43020100448517806


In [18]:
# 0.4302010044852067
# 0.43020100448499854
if new_r2 < r2:
    print('Качество ухучшилось')
else:
    print('Качество не ухучшилось')

Качество ухучшилось


**Вывод**

Умножив признаки на рандомную нормальную матрицу, информация о пользователях измиенилась, но качество линейной регрессии не ухучшилось, следовательно алгоритм подобран правильно.